In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    #input_index = np.array(range(0, length, 24))
    input_index = np.array(range(inputs_day.shape[0]))
    #label_index = input_index + 24* (7 + predict_day)
    label_index = input_index + (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    o_ = np.mean(label_day[:,predict_day,:])
    p_ = np.mean(pred_day[:,predict_day,:])
    
    oi = label_day[:,predict_day,:]
    pi = pred_day[:,predict_day,:]
    
    high = ((oi-o_)*(pi-p_)).sum()
    low = np.sqrt( ( (oi-o_)**2 ).sum() )
    low = low * np.sqrt( ( (pi-p_)**2 ).sum() )
    
    R = high/low
    RS = (R)**2
    
    return nse3, pbias3, pred_day, labels, mae, rmse, RS, R

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e) 


1 Physical GPUs, 1 Logical GPUs


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters['watershed']

'han_pull'

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 15)

In [12]:
#run_num = range(len(folder))
run_num = [0]
#if file_parameters['watershed'] == 'nak':
#    run_num = range(len(folder))
#else:
#    run_num = [0]



real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

ddday = 31
mmmonth = 12

#length = 3

In [ ]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])
    df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
                               colum_idx[idx], interpolation=interpolation_option[idx],
                              first_file_no=i, month=mmmonth, day=ddday)

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/han/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['가평_2016.xlsx', '가평_2017.xlsx', '가평_2018.xlsx'], ['서상_2016.xlsx', '서상_2017.xlsx', '서상_2018.xlsx'], ['의암호_2016.xlsx', '의암호_2017.xlsx', '의암호_2018.xlsx']]
Epoch 1/2000
1/1 [==============================] - 0s 195ms/step - gen_loss: 118.7040 - disc_loss: 0.7253 - rmse: 0.9848 - val_loss: 0.9713
Epoch 2/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 90.1198 - disc_loss: 0.5638 - rmse: 0.8793 - val_loss: 0.8619
Epoch 3/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 82.9069 - disc_loss: 0.4619 - rmse: 0.8569 - val_loss: 0.8190
Epoch 4/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 70.2316 - disc_loss: 0.4175 - rmse: 0.7932 - val_loss: 0.7397
Epoch 5/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 66.7489 - disc_loss: 0.3908 - rmse: 0.7601 - val_loss: 0.7282
Epoch 6/2000
1/1 [================

1/1 [==============================] - 0s 20ms/step - gen_loss: 8.2488 - disc_loss: 0.3125 - rmse: 0.3226 - val_loss: 0.3438
Epoch 58/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.2483 - disc_loss: 0.3179 - rmse: 0.3360 - val_loss: 0.3778
Epoch 59/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 9.0470 - disc_loss: 0.3139 - rmse: 0.3391 - val_loss: 0.3498
Epoch 60/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 9.8459 - disc_loss: 0.3051 - rmse: 0.3698 - val_loss: 0.3136
Epoch 61/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.9650 - disc_loss: 0.3137 - rmse: 0.3549 - val_loss: 0.3522
Epoch 62/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.6860 - disc_loss: 0.2932 - rmse: 0.3298 - val_loss: 0.2978
Epoch 63/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.9847 - disc_loss: 0.3151 - rmse: 0.3663 - val_loss: 0.4401
Epoch 64/2000
1/1 [======================

Epoch 116/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.6955 - disc_loss: 0.2499 - rmse: 0.3399 - val_loss: 0.3432
Epoch 117/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.9342 - disc_loss: 0.2578 - rmse: 0.3288 - val_loss: 0.3777
Epoch 118/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.7339 - disc_loss: 0.2648 - rmse: 0.3020 - val_loss: 0.3646
Epoch 119/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.4667 - disc_loss: 0.2533 - rmse: 0.2955 - val_loss: 0.3242
Epoch 120/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.5567 - disc_loss: 0.2520 - rmse: 0.3025 - val_loss: 0.2988
Epoch 121/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.2377 - disc_loss: 0.2727 - rmse: 0.2791 - val_loss: 0.3230
Epoch 122/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.1746 - disc_loss: 0.2531 - rmse: 0.3264 - val_loss: 0.3610
Epoch 123/2000
1/1 [

1/1 [==============================] - 0s 20ms/step - gen_loss: 4.5448 - disc_loss: 0.2353 - rmse: 0.2446 - val_loss: 0.2746
Epoch 175/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.4261 - disc_loss: 0.2317 - rmse: 0.3022 - val_loss: 0.3127
Epoch 176/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.2709 - disc_loss: 0.2376 - rmse: 0.2738 - val_loss: 0.2751
Epoch 177/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 12.3807 - disc_loss: 0.2319 - rmse: 0.3062 - val_loss: 0.2428
Epoch 178/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.1675 - disc_loss: 0.2330 - rmse: 0.2766 - val_loss: 0.2947
Epoch 179/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.2731 - disc_loss: 0.2394 - rmse: 0.3067 - val_loss: 0.3007
Epoch 180/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.4848 - disc_loss: 0.2333 - rmse: 0.2801 - val_loss: 0.3070
Epoch 181/2000
1/1 [==============

1/1 [==============================] - 0s 20ms/step - gen_loss: 4.0975 - disc_loss: 0.2148 - rmse: 0.2494 - val_loss: 0.2848
Epoch 233/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.2352 - disc_loss: 0.2149 - rmse: 0.2430 - val_loss: 0.2987
Epoch 234/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.5627 - disc_loss: 0.2219 - rmse: 0.2346 - val_loss: 0.3003
Epoch 235/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.6812 - disc_loss: 0.2152 - rmse: 0.2883 - val_loss: 0.3285
Epoch 236/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.4468 - disc_loss: 0.2340 - rmse: 0.2801 - val_loss: 0.2715
Epoch 237/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.0881 - disc_loss: 0.2158 - rmse: 0.2518 - val_loss: 0.3109
Epoch 238/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.9115 - disc_loss: 0.2317 - rmse: 0.2635 - val_loss: 0.2633
Epoch 239/2000
1/1 [===============

...

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6913 - disc_loss: 0.1486 - rmse: 0.1996 - val_loss: 0.2432
Epoch 1103/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9413 - disc_loss: 0.1603 - rmse: 0.2006 - val_loss: 0.2320
Epoch 1104/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1139 - disc_loss: 0.1606 - rmse: 0.1862 - val_loss: 0.2182
Epoch 1105/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8751 - disc_loss: 0.1545 - rmse: 0.2320 - val_loss: 0.2055
Epoch 1106/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.0601 - disc_loss: 0.1621 - rmse: 0.2769 - val_loss: 0.2274
Epoch 1107/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.1525 - disc_loss: 0.1668 - rmse: 0.2584 - val_loss: 0.2652
Epoch 1108/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7035 - disc_loss: 0.1700 - rmse: 0.2664 - val_loss: 0.2158
Epoch 1109/2000
1/1 [=======

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2528 - disc_loss: 0.1543 - rmse: 0.2290 - val_loss: 0.2323
Epoch 1161/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1554 - disc_loss: 0.1647 - rmse: 0.2244 - val_loss: 0.2833
Epoch 1162/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.2652 - disc_loss: 0.1581 - rmse: 0.2365 - val_loss: 0.2744
Epoch 1163/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.5538 - disc_loss: 0.1730 - rmse: 0.2856 - val_loss: 0.2540
Epoch 1164/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0145 - disc_loss: 0.1614 - rmse: 0.2475 - val_loss: 0.2900
Epoch 1165/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.3351 - disc_loss: 0.1480 - rmse: 0.2753 - val_loss: 0.2188
Epoch 1166/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5389 - disc_loss: 0.1655 - rmse: 0.2056 - val_loss: 0.2370
Epoch 1167/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8931 - disc_loss: 0.1595 - rmse: 0.2517 - val_loss: 0.1918
Epoch 1219/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.5094 - disc_loss: 0.1704 - rmse: 0.2130 - val_loss: 0.2204
Epoch 1220/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2623 - disc_loss: 0.1664 - rmse: 0.2153 - val_loss: 0.2369
Epoch 1221/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1274 - disc_loss: 0.1560 - rmse: 0.2002 - val_loss: 0.2488
Epoch 1222/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.7625 - disc_loss: 0.1470 - rmse: 0.1776 - val_loss: 0.1868
Epoch 1223/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3028 - disc_loss: 0.1521 - rmse: 0.2013 - val_loss: 0.2404
Epoch 1224/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4643 - disc_loss: 0.1508 - rmse: 0.2457 - val_loss: 0.2565
Epoch 1225/2000
1/1 [========

1/1 [==============================] - 0s 148ms/step - gen_loss: 3.3120 - disc_loss: 0.1532 - rmse: 0.2414 - val_loss: 0.2070
Epoch 1277/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1896 - disc_loss: 0.1650 - rmse: 0.2612 - val_loss: 0.3062
Epoch 1278/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2072 - disc_loss: 0.1654 - rmse: 0.2379 - val_loss: 0.2775
Epoch 1279/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3523 - disc_loss: 0.1571 - rmse: 0.2337 - val_loss: 0.2100
Epoch 1280/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2333 - disc_loss: 0.1722 - rmse: 0.2507 - val_loss: 0.2266
Epoch 1281/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8063 - disc_loss: 0.1687 - rmse: 0.2389 - val_loss: 0.2150
Epoch 1282/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2259 - disc_loss: 0.1637 - rmse: 0.2235 - val_loss: 0.2655
Epoch 1283/2000
1/1 [=======

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1257 - disc_loss: 0.1476 - rmse: 0.2116 - val_loss: 0.2313
Epoch 1335/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9632 - disc_loss: 0.1699 - rmse: 0.2093 - val_loss: 0.2375
Epoch 1336/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3551 - disc_loss: 0.1511 - rmse: 0.2449 - val_loss: 0.2227
Epoch 1337/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.4759 - disc_loss: 0.1515 - rmse: 0.2757 - val_loss: 0.1936


In [ ]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [ ]:

#pd.DataFrame(gan).tail
#real_df_all.tail



In [ ]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

In [ ]:
#ctrl f -- 타겟

print(real_df_all.shape)
rnn_target_column = 'tn'
print(rnn_target_column)



In [ ]:

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

train_df, test_df, val_df = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, val=1, test=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

In [ ]:
#fake_df.shape[0]/8760

In [ ]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)


In [ ]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

In [ ]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df)

model_path = "save/" + watershed + "models/" + file_parameters['watershed'] + '/'+  pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    if not os.path.exists(model_path):
        os.makedirs(model_path)

In [ ]:
rnn_epochs, rnn_in_setps

## 모델 학습

In [ ]:
gru_model = model_gru(
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15, steps_per_epoch = 5,
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=2, steps_per_epoch = 2,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=False)
    #training_flag=True, checkpoint_path=model_path+"gru.ckpt", continue_train=False)

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=real_df_all, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)
    
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column , " ", out_features[0])
print("length : ",real_df_all.shape[0])
print(teg_check)

print('all_nse : ', val_nse['GRU'])
print('all_pbias : ', val_pbias['GRU'])
print('all_mae : ', _mae)
print('all_rmse : ', _rmse)
print('all_Rs : ', _Rs)
print('all_R : ', _R)


In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=train_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)

print('----------------------------')
print('train_nse : ', val_nse['GRU'])
print('train_pbias : ', val_pbias['GRU'])
print('train_mae : ', _mae)
print('train_rmse : ', _rmse)
print('train_Rs : ', _Rs)
print('train_R : ', _R)

#print('nse_train : ', val_nse['GRU'])

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)

print('----------------------------')
print('val_nse : ', val_nse['GRU'])
print('val_pbias : ', val_pbias['GRU'])
print('val_mae : ', _mae)
print('val_rmse : ', _rmse)
print('val_Rs : ', _Rs)
print('val_R : ', _R)

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)

print('----------------------------')
print('test_nse : ', val_nse['GRU'])
print('test_pbias : ', val_pbias['GRU'])
print('test_mae : ', _mae)
print('test_rmse : ', _rmse)
print('test_Rs : ', _Rs)
print('test_R : ', _R)

In [ ]:
#train_df.shape, val_df.shape, test_df.shape

In [ ]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvb111nfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************       (*_*)  
#                   SSSSSSSS               ******            ******              *****    *****            ~( (>
#        SSSS         S  SSS               ******            ******              *****    *****            / (          
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

In [ ]:

sub_flag = True
history_nse = []

bob = 0
b = 1

while sub_flag:
    

    
    gru_model = model_gru(
        #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
        window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15, steps_per_epoch = 5,
        training_flag=True, checkpoint_path=model_path+"gru.ckpt", continue_train=False)
    
    #nse, _, _, _ = compa(
        #model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
        #predict_day = rnn_predict_day)
        #predict_day = 4)
    
    nse, _, _, _, _, _, _, _ = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
    
    if(nse > 0.5):
        sub_flag = False
   
    
    history_nse.append(nse)
    
    print('--------------------------')
    print(history_nse)
    print(nse)
    print('--------------------------')
    
        
    bob += b
    if(bob < -4):
        b = 1
    if(bob > 0):
        b = -1

